***k-means++算法寻找预选框

In [4]:
from scipy.io import loadmat, savemat
import matplotlib.pyplot as plt 
import os
import shutil
import random
import numpy as np
import torch
import cv2 as cv

In [14]:
class kmeans_plus():
    def __init__(self,
                 epoch = 10,
                 K = 6,
                 threshold = 0.1
                ):
        self.epoch = epoch
        self.threshold = threshold
        self.K = K
        self.m = loadmat("./wider_train/train.mat")
        self.data = self.m['face_bbx_list']
        self.Data = []
        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        for i in range(len(self.data)):
            for j in range(len(self.data[i][0])):
                for k in range(len(self.data[i][0][j][0])):
                    self.Data.append(self.data[i][0][j][0][k])
    
    def IOU(self, box1, box2):
        """
        可以广播
        Returns the IoU of two bounding boxes 
        得到bbox的坐标
        """
        # Get the coordinates of bounding boxes
        b1_x1, b1_y1, b1_x2, b1_y2 = box1[:, 0], box1[:, 1], box1[:, 2], box1[:, 3]
        b2_x1, b2_y1, b2_x2, b2_y2 = box2[:, 0], box2[:, 1], box2[:, 2], box2[:, 3]

        # get the corrdinates of the intersection rectangle
        inter_rect_x1 = torch.max(b1_x1, b2_x1)
        inter_rect_y1 = torch.max(b1_y1, b2_y1)
        inter_rect_x2 = torch.min(b1_x2, b2_x2)
        inter_rect_y2 = torch.min(b1_y2, b2_y2)

        # Intersection area
        if torch.cuda.is_available():
            inter_area = torch.max(inter_rect_x2 - inter_rect_x1 + 1, torch.zeros(inter_rect_x2.shape).cuda()) * torch.max(
                inter_rect_y2 - inter_rect_y1 + 1, torch.zeros(inter_rect_x2.shape).cuda())
        else:
            inter_area = torch.max(inter_rect_x2 - inter_rect_x1 + 1, torch.zeros(inter_rect_x2.shape)) * torch.max(
                inter_rect_y2 - inter_rect_y1 + 1, torch.zeros(inter_rect_x2.shape))

        # Union Area
        b1_area = (b1_x2 - b1_x1 + 1) * (b1_y2 - b1_y1 + 1)
        b2_area = (b2_x2 - b2_x1 + 1) * (b2_y2 - b2_y1 + 1)

        iou = inter_area / (b1_area + b2_area - inter_area)

        return iou
    
    def train(self):
        data = torch.Tensor(self.Data)
        data[:,0:2] = 0#将数据的左上坐标移动到0,0，因为我们只要得到预选框的大小和比例
        data = data.tolist()
        random.shuffle(data)
        for epoch in range(self.epoch):
            if epoch==0:
                '''
                epoch=0时采用k-means++初始化方法
                '''
                center = []
                for i in range(self.K):
                    if i == 0:
                        center.append(data[i])
                        del data[i]
                    else:
                        Diou = torch.zeros(100)
                        select = torch.Tensor(data[i*100:(i+1)*100])
                        for box in center:
                            Diou += (1-self.IOU(torch.Tensor([box]), select))
                        index = Diou.argmax()
                        center.append(data[i*100+index])
                        del data[i*100+index]
            else:
                '''
                当epoch!=0时开始训练
                很具距离对data中的box进行分类，然后重新选取中心center
                '''
                D = []
                center_new = []
                for box in center:
                    Diou = (1-self.IOU(torch.Tensor([box]), torch.Tensor(data)))
                    D.append(Diou.tolist())
                D = torch.Tensor(D)
                index = D.argmin(0)
                c_list = [[] for i in range(self.K)]
                for i,k in enumerate(index):
                    c_list[k].append(data[i])#将各个分类的box qppend在指定index的list下
                for i in c_list:
                    i = torch.Tensor(i)
                    center_new.append(i.mean(0).tolist())
                #print('center_s :  ',center)
                #print('center_new :',center_new)
                Difference = 1 - self.IOU(torch.Tensor(center),torch.Tensor(center_new)).mean()
                print('Difference :',Difference)
                if Difference<self.threshold:#判别算法的收敛
                    return center_new 
                    break
                else:
                    center = center_new

***开始训练

In [15]:
k = kmeans_plus(epoch = 1000,
                K = 25,
                threshold = 0.0001
               )

In [16]:
center = k.train()

Difference : tensor(0.1069)
Difference : tensor(0.0617)
Difference : tensor(0.0460)
Difference : tensor(0.0353)
Difference : tensor(0.0255)
Difference : tensor(0.0213)
Difference : tensor(0.0142)
Difference : tensor(0.0137)
Difference : tensor(0.0121)
Difference : tensor(0.0116)
Difference : tensor(0.0121)
Difference : tensor(0.0094)
Difference : tensor(0.0103)
Difference : tensor(0.0110)
Difference : tensor(0.0061)
Difference : tensor(0.0058)
Difference : tensor(0.0098)
Difference : tensor(0.0132)
Difference : tensor(0.0115)
Difference : tensor(0.0065)
Difference : tensor(0.0069)
Difference : tensor(0.0085)
Difference : tensor(0.0090)
Difference : tensor(0.0064)
Difference : tensor(0.0042)
Difference : tensor(0.0036)
Difference : tensor(0.0030)
Difference : tensor(0.0028)
Difference : tensor(0.0023)
Difference : tensor(0.0022)
Difference : tensor(0.0021)
Difference : tensor(0.0028)
Difference : tensor(0.0042)
Difference : tensor(0.0042)
Difference : tensor(0.0039)
Difference : tensor(

***对预选框按照面积排序

In [18]:
def Key(x):
    return x[2]*x[3]
center.sort(key = Key)
center

[[0.0, 0.0, 1.9979231357574463, 2.1740126609802246],
 [0.0, 0.0, 2.878885269165039, 3.2853729724884033],
 [0.0, 0.0, 3.983743906021118, 4.229082107543945],
 [0.0, 0.0, 4.490837574005127, 5.660663604736328],
 [0.0, 0.0, 6.171291351318359, 6.180200576782227],
 [0.0, 0.0, 5.667664527893066, 8.211201667785645],
 [0.0, 0.0, 7.979061603546143, 8.73978328704834],
 [0.0, 0.0, 8.327091217041016, 13.036238670349121],
 [0.0, 0.0, 10.593515396118164, 10.945464134216309],
 [0.0, 0.0, 12.808866500854492, 14.513771057128906],
 [0.0, 0.0, 17.269454956054688, 17.22980308532715],
 [0.0, 0.0, 14.225340843200684, 21.617671966552734],
 [0.0, 0.0, 20.769611358642578, 22.505247116088867],
 [0.0, 0.0, 24.10456657409668, 29.25560760498047],
 [0.0, 0.0, 31.61034393310547, 32.94377899169922],
 [0.0, 0.0, 35.333595275878906, 44.258445739746094],
 [0.0, 0.0, 46.83838653564453, 50.70481872558594],
 [0.0, 0.0, 51.26889419555664, 68.76127624511719],
 [0.0, 0.0, 73.70413970947266, 68.53812408447266],
 [0.0, 0.0, 70.82